In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ".json"

import vertexai
from vertexai.preview import rag

vertexai.init(project="proyecto", location="us-east4")
print(list(rag.list_corpora()))

In [ ]:
!pip install pandas

In [ ]:
# =========================================================
# Clasificación de sentencias (resultado & parte demandada) con Vertex AI
# - Lee .txt desde GCS bucket: 
# - Guarda CSVs en GCS bucket: 
# - Reusa UI, logging y helpers
# - Crea: gs:///sentencias_resultado_vertex.csv (nombre, demandado, resultado)
# - Fusiona con gs:///sentencias_motivos_vertex.csv (si existe) en gs:///sentencias_merged_vertex.csv
# =========================================================
import os, re, json, time, csv, unicodedata, textwrap, tempfile, io
from typing import Any, Dict, List, Tuple, Optional

import pandas as pd
from IPython.display import display, clear_output, Markdown

# ---- Google Cloud ----
from google.cloud import storage

# ---- Vertex AI ----
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

# -------------------- CONFIG --------------------
PROJECT_ID = "proyectid"
LOCATION   = "us-east4"
MODEL_ID   = "gemini-2.0-flash"      # o "gemini-2.0-pro"

# Buckets y nombres de objetos
SRC_BUCKET = "bucket"   # de aquí vienen los .txt
DST_BUCKET = "bucket2"        # aquí guardamos los CSVs

GCS_MOTIVOS_IN = "sentencias_motivos_vertex.csv"         # si existe en curated5896, se usa para merge
GCS_OUT_RES    = "sentencias_resultado_vertex.csv"       # salida
GCS_OUT_MERGED = "sentencias_merged_vertex.csv"          # salida fusionada

# Log local (opcional)
LOG_PATH       = "/dataset/log_vertex.txt"

# Catálogo fijo (del script original)
MOTIVOS = [
    "acreditación de la deuda",
    "usura",
    "abusividad del clausulado",
    "transparencia",
    "legitimación activa",
    "prescripción",
    "validez de la firma",
    "legitimación pasiva",
    "requerimiento previo derecho al honor",
]

# UI
VERBOSE_UI = True
UI_WRAP = 110
RAW_UI_MAX = 2500  

# -------------------- UI dinámica --------------------
def _bar(p: float, width: int = 28) -> str:
    p = min(max(p, 0.0), 1.0)
    filled = int(round(p * width))
    return "█" * filled + "░" * (width - filled)

class LiveUI:
    def __init__(self, enabled=True): self.enabled = enabled
    def _truncate(self, s, max_chars=1200): s = s or "";  return (s[:max_chars] + " …[+trunc]") if len(s) > max_chars else s
    def show(self, archivo, fase, detalle=None, ultimo_json=None, ultimo_raw=None,
             progress=None):
        if not self.enabled: return
        clear_output(wait=True)
        lines = []
        lines.append(f"**📄 Archivo:** `{archivo}`")
        lines.append(f"**⚙️ Fase:** {fase}")
        if progress is not None:
            percent = int(progress * 100)
            lines.append(f"**Progreso:** `{percent:>3d}%`  `{_bar(progress)}`")
        if detalle:
            lines.append(f"**ℹ️** {detalle}")
        if ultimo_json:
            wrapped = "\n".join(textwrap.wrap(self._truncate(ultimo_json, 2000), width=UI_WRAP))
            lines.append("**🧠 Respuesta (JSON limpio):**\n\n```json\n" + wrapped + "\n```")
        if ultimo_raw is not None:
            show = ultimo_raw if RAW_UI_MAX is None or len(ultimo_raw) <= RAW_UI_MAX else (ultimo_raw[:RAW_UI_MAX] + " …[+trunc]")
            lines.append("**📜 Respuesta LITERAL del modelo:**\n\n```\n" + show + "\n```")
        display(Markdown("\n\n".join(lines)))

ui = LiveUI(enabled=VERBOSE_UI)

# -------------------- Utilidades --------------------
def normalize_ws(s: str) -> str:
    return re.sub(r"[ \t]+\n", "\n", re.sub(r"[ \t]{2,}", " ", s or "")).strip()

# -------------------- CSV + LOG --------------------
def log_block(path, header, prompt_full, response_full):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    ts = time.strftime("%Y-%m-%d %H:%M:%S")
    with open(path, "a", encoding="utf-8") as f:
        f.write(f"{ts} | {header}\n")
        f.write("--- PROMPT START ---\n")
        f.write((prompt_full or "") + "\n")
        f.write("--- PROMPT END ---\n")
        f.write("--- RESPONSE START ---\n")
        f.write((response_full or "") + "\n")
        f.write("--- RESPONSE END ---\n")

# -------------------- GCS helpers --------------------
_storage_client = storage.Client()

def gcs_list_txt(bucket_name: str, prefix: Optional[str] = None) -> List[str]:
    """Lista blobs .txt en un bucket opcionalmente bajo un prefijo."""
    bucket = _storage_client.bucket(bucket_name)
    blobs = _storage_client.list_blobs(bucket, prefix=prefix)
    names = [b.name for b in blobs if b.name.lower().endswith(".txt")]
    names.sort()
    return names

def gcs_read_text(bucket_name: str, blob_name: str, encoding: str = "utf-8") -> str:
    """Descarga un blob de texto como string."""
    bucket = _storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    return blob.download_as_text(encoding=encoding)

def gcs_blob_exists(bucket_name: str, blob_name: str) -> bool:
    bucket = _storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    return blob.exists()

def gcs_upload_bytes(bucket_name: str, blob_name: str, data: bytes, content_type: str = "text/plain"):
    bucket = _storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_string(data, content_type=content_type)

def gcs_upload_file(bucket_name: str, blob_name: str, local_path: str, content_type: Optional[str] = None):
    bucket = _storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(local_path, content_type=content_type)

def gcs_download_to_temp(bucket_name: str, blob_name: str) -> str:
    """Descarga un blob a un fichero temporal y devuelve su ruta."""
    bucket = _storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    _, tmp_path = tempfile.mkstemp(prefix="gcs_", suffix=os.path.splitext(blob_name)[1])
    blob.download_to_filename(tmp_path)
    return tmp_path

# -------------------- Prompt + Schema (RESULTADO/DEMANDADO) --------------------
def build_prompt_outcome(texto: str) -> str:
    return f"""Eres un analista jurídico estricto.
Lee la SENTENCIA COMPLETA en español y devuelve EXCLUSIVAMENTE un JSON con DOS claves:
- "demandado": uno de ["xxx","contrario"] {{"xxx" si xxx (o xxx) es la parte DEMANDADA; "contrario" si la parte demandada es otra distinta a xxx}}
- "resultado": uno de ["favorable","desfavorable"] {{"favorable" si el FALLO es favorable a xxx; "desfavorable" si es favorable a la otra parte}}

Reglas:
- Responde en JSON VÁLIDO (RFC 8259) y NADA MÁS.
- Si el texto no identifica con claridad a la parte demandada o el fallo, infiere con criterio jurídico (fundamentos y fallo).
- Si mencionan "Se estima la demanda de xxx" → resultado="favorable".
- Si "Se desestima la demanda de xxx" → resultado="desfavorable".
- Si xxx es actor y gana, el fallo también es favorable a xxx.
- Si xxx es demandado y se desestima la demanda del actor, también es favorable a xxx.

Ejemplo de salida:
{{"demandado":"xxx","resultado":"favorable"}}

TEXTO COMPLETO:
\"\"\"{normalize_ws(texto)}\"\"\""""

RESPONSE_SCHEMA_OUTCOME: Dict[str, Any] = {
    "type": "object",
    "properties": {
        "demandado": {"type": "string", "enum": ["xxx", "contrario"]},
        "resultado": {"type": "string", "enum": ["favorable", "desfavorable"]},
    },
    "required": ["demandado", "resultado"]
}

# -------------------- Llamada a Vertex (reutilizable) --------------------
def vertex_generate_json(prompt: str,
                         model_id: str = MODEL_ID,
                         temp_primary: float = 0.0,
                         temp_fallback: float = 0.2,
                         tok_primary: int = 2048,
                         tok_fallback: int = 3072) -> str:
    model = GenerativeModel(model_id)
    last_err = None
    # 1) JSON mime con temp baja
    try:
        cfg = GenerationConfig(
            response_mime_type="application/json",
            temperature=temp_primary,
            max_output_tokens=tok_primary,
        )
        r = model.generate_content([prompt], generation_config=cfg)
        raw = (r.text or "").strip()
        if raw:
            return raw
    except Exception as e1:
        last_err = e1
    # 2) JSON mime fallback
    try:
        cfg2 = GenerationConfig(
            response_mime_type="application/json",
            temperature=temp_fallback,
            max_output_tokens=tok_fallback,
        )
        r2 = model.generate_content([prompt], generation_config=cfg2)
        raw2 = (r2.text or "").strip()
        if raw2:
            return raw2
    except Exception as e2:
        last_err = e2
    # 3) Libre
    r3 = model.generate_content(
        [prompt],
        generation_config=GenerationConfig(
            temperature=temp_fallback,
            max_output_tokens=tok_fallback
        )
    )
    raw3 = (r3.text or "").strip()
    if raw3:
        return raw3
    raise RuntimeError(f"Vertex devolvió vacío: {last_err if last_err else 'sin detalle'}")

def vertex_generate_json_with_schema(prompt: str, schema: Dict[str, Any],
                                     model_id: str = MODEL_ID,
                                     temp: float = 0.0,
                                     max_tokens: int = 2048) -> str:
    model = GenerativeModel(model_id)
    r = model.generate_content(
        [prompt],
        generation_config=GenerationConfig(
            response_mime_type="application/json",
            response_schema=schema,
            temperature=temp,
            max_output_tokens=max_tokens
        )
    )
    return (r.text or "").strip()

# -------------------- Parser genérico --------------------
def extract_first_json_value(s: str) -> Optional[str]:
    if not s: return None
    s = s.strip()
    s = re.sub(r'^```(?:json)?\s*', '', s, flags=re.IGNORECASE | re.MULTILINE)
    s = re.sub(r'\s*```$', '', s, flags=re.IGNORECASE)
    starts = [i for i, ch in enumerate(s) if ch in "{["]
    if not starts: return None
    start = starts[0]
    open_ch = s[start]; close_ch = "}" if open_ch == "{" else "]"
    depth = 0
    for i in range(start, len(s)):
        ch = s[i]
        if ch == open_ch: depth += 1
        elif ch == close_ch:
            depth -= 1
            if depth == 0: return s[start:i+1]
    return None

# --- Parse motivos (por si lo necesitas en el merged) ---
def parsear_motivos_desde_obj(obj) -> List[Tuple[str, float]]:
    arr = None
    if isinstance(obj, dict):
        if "motivos" in obj: arr = obj["motivos"]
    elif isinstance(obj, list):
        arr = obj
    out = []
    if not isinstance(arr, list): return out
    for item in arr:
        if isinstance(item, str):
            canon = canonizar_motivo(item)
            if canon: out.append((canon, 1.0))
        elif isinstance(item, dict):
            m = item.get("motivo"); c = item.get("confianza", 1.0)
            canon = canonizar_motivo(m) if m else None
            if canon:
                try: c = float(c)
                except: c = 1.0
                out.append((canon, max(0.0, min(1.0, c))))
    return out

# -------------------- Clasificación OUTCOME por sentencia --------------------
def clasificar_outcome_fulltext(nombre_archivo: str, texto: str) -> Tuple[str, str]:
    prompt = build_prompt_outcome(texto)
    # 1) Intento con schema
    try:
        raw = vertex_generate_json_with_schema(prompt, RESPONSE_SCHEMA_OUTCOME,
                                               model_id=MODEL_ID, temp=0.0, max_tokens=1024)
    except Exception:
        # 2) Fallbacks
        raw = vertex_generate_json(prompt, model_id=MODEL_ID, temp_primary=0.0, temp_fallback=0.2)
    # LOG completo
    log_block(LOG_PATH, f"{nombre_archivo} | OUTCOME", prompt, raw)

    frag_json = extract_first_json_value(raw)
    ui_json_to_show = frag_json or "{}"
    ui.show(archivo=nombre_archivo, fase="Inferencia OUTCOME completada",
            ultimo_json=ui_json_to_show, ultimo_raw=raw, progress=1.0)

    if not frag_json:
        return ("contrario", "desfavorable")  # fallback prudente

    try:
        obj = json.loads(frag_json)
        demandado = str(obj.get("demandado", "")).strip().lower()
        resultado = str(obj.get("resultado", "")).strip().lower()
        if demandado not in ("eos", "contrario"):
            demandado = "contrario"
        if resultado not in ("favorable", "desfavorable"):
            resultado = "desfavorable"
        return (demandado, resultado)
    except Exception:
        return ("contrario", "desfavorable")

# -------------------- MAIN: lee de GCS, genera CSV en GCS y fusiona --------------------
def main(prefix: Optional[str] = None):
   
    # Inicializa Vertex
    vertexai.init(project=PROJECT_ID, location=LOCATION)

    # 1) Enumerar sentencias .txt desde GCS
    txt_blobs = gcs_list_txt(SRC_BUCKET, prefix=prefix)
    if not txt_blobs:
        print(f"⚠️ No se encontraron .txt en gs://{SRC_BUCKET}/{prefix or ''}")
        return

    total = len(txt_blobs)
    rows_outcome = []  # acumularemos aquí (nombre, demandado, resultado)

    for idx, blob_name in enumerate(txt_blobs, start=1):
        # Nombre de archivo sin extensión (para la clave 'nombre')
        base = os.path.basename(blob_name)
        nombre_sin_ext = os.path.splitext(base)[0]

        ui.show(archivo=base, fase=f"Procesando OUTCOME {idx}/{total}",
                detalle=f"Descargando texto desde gs://{SRC_BUCKET}/{blob_name}…",
                progress=(idx-1)/total)

        try:
            texto = gcs_read_text(SRC_BUCKET, blob_name)
        except Exception as e:
            ui.show(archivo=base, fase=f"ERROR descarga {idx}/{total}",
                    detalle=f"No se pudo leer el blob: {e}", progress=(idx-1)/total)
            continue

        demandado, resultado = clasificar_outcome_fulltext(base, texto)
        rows_outcome.append((nombre_sin_ext, demandado, resultado))

        ui.show(archivo=base, fase=f"Clasificado OUTCOME {idx}/{total}",
                detalle=f"{nombre_sin_ext} → demandado={demandado}, resultado={resultado}",
                progress=idx/total)

    # 2) Crear DataFrame y subir CSV de resultados al bucket destino
    df_resultado = pd.DataFrame(rows_outcome, columns=["nombre", "demandado", "resultado"])
    with tempfile.NamedTemporaryFile(mode="w", newline="", suffix=".csv", delete=False, encoding="utf-8") as tmpf:
        df_resultado.to_csv(tmpf.name, index=False, encoding="utf-8")
        gcs_upload_file(DST_BUCKET, GCS_OUT_RES, tmpf.name, content_type="text/csv")

    # 3) Intentar fusionar con CSV de motivos en el mismo bucket destino
    merged_done = False
    if gcs_blob_exists(DST_BUCKET, GCS_MOTIVOS_IN):
        # Descargar motivos a temp y leer
        tmp_motivos = gcs_download_to_temp(DST_BUCKET, GCS_MOTIVOS_IN)
        df_motivos = pd.read_csv(tmp_motivos)

        merged = pd.merge(df_motivos, df_resultado, on="nombre", how="outer")

        with tempfile.NamedTemporaryFile(mode="w", newline="", suffix=".csv", delete=False, encoding="utf-8") as tmpm:
            merged.to_csv(tmpm.name, index=False, encoding="utf-8")
            gcs_upload_file(DST_BUCKET, GCS_OUT_MERGED, tmpm.name, content_type="text/csv")
        merged_done = True

    # 4) Mensaje final
    detalle = f"CSV OUTCOME → gs://{DST_BUCKET}/{GCS_OUT_RES}"
    if merged_done:
        detalle += f"\nCSV fusionado → gs://{DST_BUCKET}/{GCS_OUT_MERGED}"
    else:
        detalle += f"\n⚠️ No se encontró gs://{DST_BUCKET}/{GCS_MOTIVOS_IN}. Se omite la fusión."
    ui.show(archivo="(todos)", fase="Completado", detalle=detalle, progress=1.0)
    print(detalle)


if __name__ == "__main__":
    
    main(prefix=None)